In [ ]:
from dotenv import load_dotenv
import os 
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv('env', override=True)
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
END_POINT=os.getenv('END_POINT')
MODEL_NAME=os.getenv('MODEL_NAME')
print(AZURE_OPENAI_API_KEY[:10])
print(MODEL_NAME)

AZURE_OPENAI_EMB_API_KEY = os.getenv('AZURE_OPENAI_EMB_API_KEY')
EMB_END_POINT=os.getenv('EMB_END_POINT')
EMB_MODEL_NAME=os.getenv('EMB_MODEL_NAME')

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_TRACING_V2'] = 'true' #true, false
os.environ['LANGCHAIN_PROJECT'] = 'LANG'

if os.getenv('LANGCHAIN_TRACING_V2') == "true":
    print('랭스미스로 추적 중입니다 :', os.getenv('LANGSMITH_API_KEY')[:10])

43b13g4OZS
gpt-4.1-mini


## LCEL

LangChain Expression Language (LCEL)은 LangChain 라이브러리에서 제공하는 선언적 방식의 인터페이스

특징 : 
- 선복잡한 로직을 간결하고 읽기 쉬움
- 다양한 컴포넌트를 쉽게 조합하고 재사용
- 다양한 유형의 LLM 애플리케이션 구축
- 사용자 정의 컴포넌트를 쉽게 통합

### 1) RunnablePassthrough

입력을 그대로 출력하고 아무 처리도 안 합니다.

체인의 중간에 변수를 바로 넘길 때 사용합니다.

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = "이 질문에 간단하게 답변해줘: [질문] - {question}"
prompt = ChatPromptTemplate.from_template(template)
print(prompt)

input_variables=['question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='이 질문에 간단하게 답변해줘: [질문] - {question}'), additional_kwargs={})]


In [6]:
pt = {"question": RunnablePassthrough()} | prompt 

print(pt.invoke("내일 해는 어디에서 뜨지?"))

messages=[HumanMessage(content='이 질문에 간단하게 답변해줘: [질문] - 내일 해는 어디에서 뜨지?', additional_kwargs={}, response_metadata={})]


### 2) RunnableLambda

임의의 파이썬 함수(콜러블)를 Runnable로 감싸서 러너블화 시킴

--> 랭체인에서 LCEL로 쉽게 사용가능

In [7]:
from langchain_core.runnables import RunnableLambda

double = RunnableLambda(lambda x: x + x)
print(double.invoke(5))

10


In [8]:
def say_hello(name: str) -> str:
    return f"Hello, {name}!"

hello_runnable = RunnableLambda(say_hello)
print(hello_runnable.invoke("Alice"))

Hello, Alice!


### 3) RunnableParallel

여러 Runnable을 병렬로 실행해서 결과를 딕셔너리로 반환합니다.

같은 입력을 서로 다른 체인/함수에 동시에 넣고, 결과를 한꺼번에 받고 싶을 때 사용합니다.

In [14]:
from langchain_core.runnables import RunnableParallel

def get_customer_number(_ : str) -> str:  
    return "132"

def get_counter_number(_ : str) -> str:  
    return "4"

parallel = RunnableParallel({
    "customer_number": get_customer_number,
    "counter_number": get_counter_number,
})

print('parallel 결과')
print(parallel.invoke("Jintae"))

prompt = ChatPromptTemplate.from_template("{customer_number} 고객님 {counter_number}번 창구에 오시기 바랍니다.")

chain = parallel | prompt

print('\nchain 결과')
print(chain.invoke("Jintae"))


parallel 결과
{'customer_number': '132', 'counter_number': '4'}

chain 결과
messages=[HumanMessage(content='132 고객님 4번 창구에 오시기 바랍니다.', additional_kwargs={}, response_metadata={})]


랭그래프에서는 입력이 dict인 state가 들어오는데 itemgetter와 결합하면 편리합니다.

In [17]:
from operator import itemgetter

template = "{_customer_number} 고객님 {_counter_number}번 창구에 오시기 바랍니다."
prompt = ChatPromptTemplate.from_template(template)
chain = (
    {
        "_customer_number": itemgetter("customer_number"),
        "_counter_number": itemgetter("counter_number"),
    }
    | prompt
)

chain.invoke({"customer_number": "132", "counter_number": "4"})

ChatPromptValue(messages=[HumanMessage(content='132 고객님 4번 창구에 오시기 바랍니다.', additional_kwargs={}, response_metadata={})])

### 4) RunnableSequence

여러 Runnable을 순차적으로 연결한 파이프라인입니다.

앞선 러너블의 결과가 다음 러너블의 입력으로 들어갑니다.


In [20]:
from langchain_core.runnables import RunnableSequence

r = RunnableSequence(first=double, last=hello_runnable)
print(r.invoke('Jintae'))

# RunnableSequence 대신 파이프라인 연산자(|)를 사용할 수 있습니다.

print((double | hello_runnable).invoke('Jintae'))

Hello, JintaeJintae!
Hello, JintaeJintae!
